# Python Implementation of RSA, Digital Signature and Certificates
> Implementation of RSA, Digital Signature and Certificates without divining too deep into the Mathematics; then list out key math principles for further reference. In this way, one can understand *what* and *how* as a context before drilling into *why*.

- toc: true
- branch: master
- badges: false
- comments: true
- categories: [jupyter, python, math]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

## Introduction

Information is power. However, to transmit info is tricky. At least 2 issues need to be solved in information transmission: **Security** and **Integrity**.

- **Security (RSA)**

No one can see the naked message other than the one intended.

- **Integrity (Digital Signature)**

No one shall replace the message with another dummy one in the middle of transimission;  
If it happened, however, intended message receiver shall be able to find out.

## Rivest, Shamir and Adleman (RSA) Cryptosystem

![rsa](rsa_imgs/bg2013062702.jpg)
@MIT

### Helper Functions

In [1]:
#collapse-hide
import random
import base64
from hashlib import sha256

#### Check If a Number is Prime

In [2]:
#collapse-show
def check_prime(num):
    """Check if num is prime
    
    """
    # prime numbers are greater than 1
    if num > 1:
        # check for factors
        # TODO: refactoring to parallel
        for i in range(2, num):
            if (num % i) == 0:
                return False
        else:
            return True
    # if input number is less than
    # or equal to 1, it is not prime
    else:
        return False

#### Modular Multiplicative Inverse Function

In [3]:
#collapse-show
# https://stackoverflow.com/a/9758173/3317548
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)


def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception("modular inverse does not exist")
    else:
        aa = x % m
        bb = int((1 - a * aa) / m)
        return aa, bb

#### List All Prime Numbers Smaller Than a Value

In [4]:
#collapse-show
# https://stackoverflow.com/q/2068372/3317548
def find_prime(n):
    """ Returns  a list of primes < n """
    sieve = [True] * (n // 2)
    for i in range(3, int(n ** 0.5) + 1, 2):
        if sieve[i // 2]:
            sieve[i * i // 2 :: i] = [False] * ((n - i * i - 1) // (2 * i) + 1)
    return [2] + [2 * i + 1 for i in range(1, n // 2) if sieve[i]]

###  Encryption & Decryption Process

- Receiver (Richael) generate a pair of keys: **public** & **private**
- She keeps **private** key to herself confidentially
- She sends multiple copies of **public** key to every of her friends who need to send her message
- Any one got Richael's **public** key can send her message now. For instance, Stephen:  
    - Stephen'd like to send the original message: "Hello Richael. Let's meet at 6:25 pm tomorrow at the airport Gate5, Terminal 2 :)", to Richael.
    - Stephen'd encrypt the message with Richaels **public** key into something looks random, e.g.  "MHgxNTMzCjB4MmFmNgoweGFiOA...YWE0CjB4MWZlMwoweDIzMGY="
    - Stephen'd send that "random" stuff to Richael.
- After receiving the "random" stuff from Stephen, Richael uses her own **private** key to decrypt it into orignal message "Hello Richael. How are you?!"
- Only with Richael's **private** key, the "random" stuff, produced with Richael's **public** key, can be decrpted. Unless Richael shares her **private** key, which she shall never do, no one other than Richael herself can obtain the original message "Hello Richael. Let's meet at 6:25 pm tomorrow at the airport Gate5, Terminal 2 :)"
- If Richael'd like to answer to Stephen, she must obtain Stephen's **public** key.

#### Key Pair: Public & Private (Richael's Task)

- Randomly pick 2 prime numbers (larger the better): **p** & **q** 
- Got their muliplication: **n=pxq**
- Got the Euler number: **phi=(p-1)x(q-1)**
- Randomly find a prime number in the range of 1 to **phi** (larger the better): **e**
- Find the Modular Multiplicative Inverse of **e** relative to **phi**: **k**
- public key is: **(n, e)**
- private key is: **(n, k)**

In [5]:
# prime numbers
p = random.choice(list(find_prime(1024)))
q = random.choice(list(find_prime(1024)))
assert check_prime(p)
assert check_prime(q)
n = p * q
p, q, n, len(bin(n)), bin(n)

(941, 277, 260657, 20, '0b111111101000110001')

In [6]:
# Euler number
phi = (p - 1) * (q - 1)
phi

259440

In [7]:
e = random.choice(find_prime(phi))
assert check_prime(e)
e

96211

In [8]:
k = modinv(e, phi)[0]
k

32971

In [9]:
public_key, private_key = (n, e), (n, k)
public_key, private_key

((260657, 96211), (260657, 32971))

#### Encryption & Decryption Computation

- Original message to unicode numbers: **msg**
- Encrypt message: **msg_encrypt = (msg^e)%n**
- Decrypt message: **msg = (msg_encrypt^k)%n**

#### Origial Message (Stephen's Task)

In [10]:
#collapse-output
# unicode
msg = [ord(i) for i in "Hello Richael. Let's meet at 6:25 pm tomorrow at the airport Gate5, Terminal 2 --Stephen :)"]
msg

[72,
 101,
 108,
 108,
 111,
 32,
 82,
 105,
 99,
 104,
 97,
 101,
 108,
 46,
 32,
 76,
 101,
 116,
 39,
 115,
 32,
 109,
 101,
 101,
 116,
 32,
 97,
 116,
 32,
 54,
 58,
 50,
 53,
 32,
 112,
 109,
 32,
 116,
 111,
 109,
 111,
 114,
 114,
 111,
 119,
 32,
 97,
 116,
 32,
 116,
 104,
 101,
 32,
 97,
 105,
 114,
 112,
 111,
 114,
 116,
 32,
 71,
 97,
 116,
 101,
 53,
 44,
 32,
 84,
 101,
 114,
 109,
 105,
 110,
 97,
 108,
 32,
 50,
 32,
 45,
 45,
 83,
 116,
 101,
 112,
 104,
 101,
 110,
 32,
 58,
 41]

#### Encrption (Stephen's Task)

In [11]:
#collapse-output
# encrption
[(val ** public_key[1]) % public_key[0] for val in msg]

[194726,
 73146,
 72544,
 72544,
 9248,
 92741,
 238646,
 85607,
 200450,
 48424,
 239858,
 73146,
 72544,
 91511,
 92741,
 212777,
 73146,
 65211,
 109261,
 61397,
 92741,
 74651,
 73146,
 73146,
 65211,
 92741,
 239858,
 65211,
 92741,
 78010,
 231217,
 128639,
 171443,
 92741,
 222131,
 74651,
 92741,
 65211,
 9248,
 74651,
 9248,
 236804,
 236804,
 9248,
 164721,
 92741,
 239858,
 65211,
 92741,
 65211,
 48424,
 73146,
 92741,
 239858,
 85607,
 236804,
 222131,
 9248,
 236804,
 65211,
 92741,
 154298,
 239858,
 65211,
 73146,
 171443,
 243311,
 92741,
 38634,
 73146,
 236804,
 74651,
 85607,
 230237,
 239858,
 72544,
 92741,
 128639,
 92741,
 95148,
 95148,
 37338,
 65211,
 73146,
 222131,
 48424,
 73146,
 230237,
 92741,
 231217,
 92222]

In [12]:
# hex encode or something else
encry_msg = '\n'.join([hex((val ** public_key[1]) % public_key[0]) for val in msg])
encry_msg

'0x2f8a6\n0x11dba\n0x11b60\n0x11b60\n0x2420\n0x16a45\n0x3a436\n0x14e67\n0x30f02\n0xbd28\n0x3a8f2\n0x11dba\n0x11b60\n0x16577\n0x16a45\n0x33f29\n0x11dba\n0xfebb\n0x1aacd\n0xefd5\n0x16a45\n0x1239b\n0x11dba\n0x11dba\n0xfebb\n0x16a45\n0x3a8f2\n0xfebb\n0x16a45\n0x130ba\n0x38731\n0x1f67f\n0x29db3\n0x16a45\n0x363b3\n0x1239b\n0x16a45\n0xfebb\n0x2420\n0x1239b\n0x2420\n0x39d04\n0x39d04\n0x2420\n0x28371\n0x16a45\n0x3a8f2\n0xfebb\n0x16a45\n0xfebb\n0xbd28\n0x11dba\n0x16a45\n0x3a8f2\n0x14e67\n0x39d04\n0x363b3\n0x2420\n0x39d04\n0xfebb\n0x16a45\n0x25aba\n0x3a8f2\n0xfebb\n0x11dba\n0x29db3\n0x3b66f\n0x16a45\n0x96ea\n0x11dba\n0x39d04\n0x1239b\n0x14e67\n0x3835d\n0x3a8f2\n0x11b60\n0x16a45\n0x1f67f\n0x16a45\n0x173ac\n0x173ac\n0x91da\n0xfebb\n0x11dba\n0x363b3\n0xbd28\n0x11dba\n0x3835d\n0x16a45\n0x38731\n0x1683e'

In [13]:
# base64 encode
encry_msg = base64.b64encode(encry_msg.encode('utf8'))
encry_msg

b'MHgyZjhhNgoweDExZGJhCjB4MTFiNjAKMHgxMWI2MAoweDI0MjAKMHgxNmE0NQoweDNhNDM2CjB4MTRlNjcKMHgzMGYwMgoweGJkMjgKMHgzYThmMgoweDExZGJhCjB4MTFiNjAKMHgxNjU3NwoweDE2YTQ1CjB4MzNmMjkKMHgxMWRiYQoweGZlYmIKMHgxYWFjZAoweGVmZDUKMHgxNmE0NQoweDEyMzliCjB4MTFkYmEKMHgxMWRiYQoweGZlYmIKMHgxNmE0NQoweDNhOGYyCjB4ZmViYgoweDE2YTQ1CjB4MTMwYmEKMHgzODczMQoweDFmNjdmCjB4MjlkYjMKMHgxNmE0NQoweDM2M2IzCjB4MTIzOWIKMHgxNmE0NQoweGZlYmIKMHgyNDIwCjB4MTIzOWIKMHgyNDIwCjB4MzlkMDQKMHgzOWQwNAoweDI0MjAKMHgyODM3MQoweDE2YTQ1CjB4M2E4ZjIKMHhmZWJiCjB4MTZhNDUKMHhmZWJiCjB4YmQyOAoweDExZGJhCjB4MTZhNDUKMHgzYThmMgoweDE0ZTY3CjB4MzlkMDQKMHgzNjNiMwoweDI0MjAKMHgzOWQwNAoweGZlYmIKMHgxNmE0NQoweDI1YWJhCjB4M2E4ZjIKMHhmZWJiCjB4MTFkYmEKMHgyOWRiMwoweDNiNjZmCjB4MTZhNDUKMHg5NmVhCjB4MTFkYmEKMHgzOWQwNAoweDEyMzliCjB4MTRlNjcKMHgzODM1ZAoweDNhOGYyCjB4MTFiNjAKMHgxNmE0NQoweDFmNjdmCjB4MTZhNDUKMHgxNzNhYwoweDE3M2FjCjB4OTFkYQoweGZlYmIKMHgxMWRiYQoweDM2M2IzCjB4YmQyOAoweDExZGJhCjB4MzgzNWQKMHgxNmE0NQoweDM4NzMxCjB4MTY4M2U='

#### Decryption (Richael's Task)

In [14]:
#collapse-output
# base64 decode and hex decode
[int(i, 16) for i in base64.b64decode(encry_msg).decode('utf8').split('\n')]

[194726,
 73146,
 72544,
 72544,
 9248,
 92741,
 238646,
 85607,
 200450,
 48424,
 239858,
 73146,
 72544,
 91511,
 92741,
 212777,
 73146,
 65211,
 109261,
 61397,
 92741,
 74651,
 73146,
 73146,
 65211,
 92741,
 239858,
 65211,
 92741,
 78010,
 231217,
 128639,
 171443,
 92741,
 222131,
 74651,
 92741,
 65211,
 9248,
 74651,
 9248,
 236804,
 236804,
 9248,
 164721,
 92741,
 239858,
 65211,
 92741,
 65211,
 48424,
 73146,
 92741,
 239858,
 85607,
 236804,
 222131,
 9248,
 236804,
 65211,
 92741,
 154298,
 239858,
 65211,
 73146,
 171443,
 243311,
 92741,
 38634,
 73146,
 236804,
 74651,
 85607,
 230237,
 239858,
 72544,
 92741,
 128639,
 92741,
 95148,
 95148,
 37338,
 65211,
 73146,
 222131,
 48424,
 73146,
 230237,
 92741,
 231217,
 92222]

In [15]:
# RSA decode
decry_msg = [
    int(i, 16) ** private_key[1] % private_key[0]
    for i in base64.b64decode(encry_msg).decode("utf8").split("\n")
]
print(decry_msg)
print("".join([chr(i) for i in decry_msg]))

[72, 101, 108, 108, 111, 32, 82, 105, 99, 104, 97, 101, 108, 46, 32, 76, 101, 116, 39, 115, 32, 109, 101, 101, 116, 32, 97, 116, 32, 54, 58, 50, 53, 32, 112, 109, 32, 116, 111, 109, 111, 114, 114, 111, 119, 32, 97, 116, 32, 116, 104, 101, 32, 97, 105, 114, 112, 111, 114, 116, 32, 71, 97, 116, 101, 53, 44, 32, 84, 101, 114, 109, 105, 110, 97, 108, 32, 50, 32, 45, 45, 83, 116, 101, 112, 104, 101, 110, 32, 58, 41]
Hello Richael. Let's meet at 6:25 pm tomorrow at the airport Gate5, Terminal 2 --Stephen :)


### Why is this secure?

![Euler](rsa_imgs/euler.png)
@Euler  
*He changed the world and he is now everywhere; Remeber Euler and everything about him, otherwise you'd fail at some stage of your life, some math exams for sure.*

- If another friend of Richael, say Tom, would like to know what Stephen sent to Richael, without asking Stephen or Richael.
- Another guy, say Joseph, was eavesdropping the network communication.
- Either Tom or Joseph would already have access to information of: 
    - **n** and **e** from public key because Richael broadcast it on the network to her friends.
    - **msg_encrypt** because Stephen put it on the network for Richael to retrieve.
- In order to decrypt **msg_encrypt**, they need **private key** **(n, k)**;
- They already have **n** so only need to figure out **k**.
- **k** was generated with **e** and **phi**.
- They know **e**. So they only need to know *phi*. 
- They need **p** and **q** to know **phi**
- They know **n** which is the multiplication of prime numbers **p** and **q**
- Tom or Joseph need to do **Prime Factorization** as **n=pxq**
- Given **n** is large enough (1024 bits in nowadays common sense), Tom and Joseph would had been long gone before a modern computer, even they have access to [Fugaku](https://en.wikipedia.org/wiki/TOP500), could manage to figure out which 2 prime numebrs Richael picked at the first place, let along about by "6:25 pm tomrrow".
- Richael may choose to update her key pairs frequently (change to different **p** and **q**) for the next message; this means Tom and Joseph can never catch up without a [Quantum Computer](https://www.newscientist.com/article/2227387-quantum-computer-sets-new-record-for-finding-prime-number-factors/) on their hands.

### Investigate and Understand Your Computer

Now if go to `~/.ssh`, you shall identify a number of different key files. Some of them are private key whilst others are public key; some are generated by OpenSSH whilst others by OpenSSL. They are also encoded by ASN.1 format among some variates.

- You can convert it into `*.PEM` file via:  

`ssh-keygen -f YOUR_KEY.pub -e -m pem > YOUR_KEY.pub.pem`

- Then inspect the content via:

```python
from Crypto.PublicKey import RSA
from base64 import b64decode

pem_key = b"MII<DO NOT SHARE PRIVATE KEY>="
key = b64decode(pem_key)
keyPriv = RSA.importKey(key)
# key now has all the components of the private
print(keyPriv.keydata)
print(keyPriv.n, keyPriv.e, keyPriv.d, keyPriv.p, keyPriv.q, keyPriv.u)

assert keyPriv.p * keyPriv.q == keyPriv.n
```

For public key `*.pub.pem`, it has only `keyPriv.n` and `keyPriv.e`.   
See: https://www.dlitz.net/software/pycrypto/api/2.6/Crypto.PublicKey.RSA-module.html

- Similarly, if you check `less ~/.ssh/known_hosts`, you shall see a number of lines of records, composing of:  

`<domain_name,ip_address> <algorithm, e.g. ssh-rsa> <public_key>`  

You can convert those `public_key` into `*.pem` like above and use the code snippet to check its content. You shall got `n` and `e`. When you send message to those servers, your message is encrypted with those `n` and `e`, and those server will use their private key to decrypt your message.

## Digital Signature

What if Joseph pretends to be Stephen, and sends Richael another message encrypted with Richaels public key, saying "Hello Richael. Let's meet at 6:25 am the day after tomorrow at the train station Gate 1 :)". How does Richael know the message is truly from Stephen, or being pretended by Joseph? 

Another case would be Joseph intercepted Stephen's **msg_encrypt**, then modified/deleted/added some characters in that *random* stuff ("MHgxNTMzCjB4MmFmNgoweGFiO>B<...YWE0CjB4MWZlMwoweDIzMGY="), before sending to Rachael for decryption? Even though he cannot decrypt the message without Richael's private key, he can still alter it. For example, hiding some information about location:

In [16]:
encry_msg_alter = base64.b64encode(
    "\n".join(base64.b64decode(encry_msg).decode("utf8").split("\n")[:33]).encode(
        "utf8"
    )
)
encry_msg_alter

b'MHgyZjhhNgoweDExZGJhCjB4MTFiNjAKMHgxMWI2MAoweDI0MjAKMHgxNmE0NQoweDNhNDM2CjB4MTRlNjcKMHgzMGYwMgoweGJkMjgKMHgzYThmMgoweDExZGJhCjB4MTFiNjAKMHgxNjU3NwoweDE2YTQ1CjB4MzNmMjkKMHgxMWRiYQoweGZlYmIKMHgxYWFjZAoweGVmZDUKMHgxNmE0NQoweDEyMzliCjB4MTFkYmEKMHgxMWRiYQoweGZlYmIKMHgxNmE0NQoweDNhOGYyCjB4ZmViYgoweDE2YTQ1CjB4MTMwYmEKMHgzODczMQoweDFmNjdmCjB4MjlkYjM='

In [17]:
# RSA decode
decry_msg = [
    int(i, 16) ** private_key[1] % private_key[0]
    for i in base64.b64decode(encry_msg_alter).decode("utf8").split("\n")
]
print(decry_msg)
print("".join([chr(i) for i in decry_msg]))

[72, 101, 108, 108, 111, 32, 82, 105, 99, 104, 97, 101, 108, 46, 32, 76, 101, 116, 39, 115, 32, 109, 101, 101, 116, 32, 97, 116, 32, 54, 58, 50, 53]
Hello Richael. Let's meet at 6:25


### Two Way RSA

Previously, we introduced only one-way RSA: everyone has Richael's **public** key. But Richael can also have others' **public** key, especially, the message sender Stephen's: 

In [18]:
# Stephen generate his key pair
p = random.choice(list(find_prime(1024)))
q = random.choice(list(find_prime(1024)))
assert check_prime(p)
assert check_prime(q)
n = p * q
p, q, n, len(bin(n)), bin(n)

# Euler number
phi = (p - 1) * (q - 1)
phi

e = random.choice(find_prime(phi))
assert check_prime(e)
e

k = modinv(e, phi)[0]
k

public_key_stephen, private_key_stephen = (n, e), (n, k)
public_key_stephen, private_key_stephen

((2701, 1993), (2701, 1177))

### Hash

Stephen will keep his **private** key to himself, but share the **public** key to his friends, including Richael. Before Stephen sends the message to Richael, he will also generate an **abstract** of the total message, e.g. with Hash:

In [19]:
signature = sha256(b"Hello Richael. Let's meet at 6:25 pm tomorrow at the airport Gate5, Terminal 2 --Stephen :)").hexdigest()
signature

'22edc929d9211d1102f7af88b85ea472c9e64629074985cba94607526dff3029'

Now Stephen will encrypt this abstract with his **private** key:

In [20]:
# hex encode or something else
encry_sig = "\n".join(
    [
        hex((ord(val) ** private_key_stephen[1]) % private_key_stephen[0])
        for val in signature
    ]
)

# base64 encode
encry_sig = base64.b64encode(encry_sig.encode("utf8"))
encry_sig

b'MHg4YjEKMHg4YjEKMHg0NGMKMHg2NAoweDlmMAoweDViMgoweDhiMQoweDViMgoweDY0CjB4NWIyCjB4OGIxCjB4MTBjCjB4MTBjCjB4NjQKMHgxMGMKMHgxMGMKMHg5MjYKMHg4YjEKMHgyZjkKMHg4NTUKMHg2MQoweDJmOQoweDFlZQoweDFlZQoweGEzZQoweDFlZQoweDY2OAoweDQ0YwoweDYxCjB4M2U5CjB4ODU1CjB4OGIxCjB4OWYwCjB4NWIyCjB4NDRjCjB4ZjQKMHgzZTkKMHhmNAoweDhiMQoweDViMgoweDkyNgoweDg1NQoweDNlOQoweDViMgoweDFlZQoweDY2OAoweDlmMAoweGEzZQoweDYxCjB4NWIyCjB4M2U5CjB4ZjQKMHg5MjYKMHg4NTUKMHg2NjgKMHg4YjEKMHhmNAoweDY0CjB4MmY5CjB4MmY5CjB4MzMKMHg5MjYKMHg4YjEKMHg1YjI='

Finally, the message ready to be shipped to Richael by Stephen becomes:

In [21]:
message_to_Richael = {"msg": encry_msg, "signature": encry_sig}
message_to_Richael

{'msg': b'MHgyZjhhNgoweDExZGJhCjB4MTFiNjAKMHgxMWI2MAoweDI0MjAKMHgxNmE0NQoweDNhNDM2CjB4MTRlNjcKMHgzMGYwMgoweGJkMjgKMHgzYThmMgoweDExZGJhCjB4MTFiNjAKMHgxNjU3NwoweDE2YTQ1CjB4MzNmMjkKMHgxMWRiYQoweGZlYmIKMHgxYWFjZAoweGVmZDUKMHgxNmE0NQoweDEyMzliCjB4MTFkYmEKMHgxMWRiYQoweGZlYmIKMHgxNmE0NQoweDNhOGYyCjB4ZmViYgoweDE2YTQ1CjB4MTMwYmEKMHgzODczMQoweDFmNjdmCjB4MjlkYjMKMHgxNmE0NQoweDM2M2IzCjB4MTIzOWIKMHgxNmE0NQoweGZlYmIKMHgyNDIwCjB4MTIzOWIKMHgyNDIwCjB4MzlkMDQKMHgzOWQwNAoweDI0MjAKMHgyODM3MQoweDE2YTQ1CjB4M2E4ZjIKMHhmZWJiCjB4MTZhNDUKMHhmZWJiCjB4YmQyOAoweDExZGJhCjB4MTZhNDUKMHgzYThmMgoweDE0ZTY3CjB4MzlkMDQKMHgzNjNiMwoweDI0MjAKMHgzOWQwNAoweGZlYmIKMHgxNmE0NQoweDI1YWJhCjB4M2E4ZjIKMHhmZWJiCjB4MTFkYmEKMHgyOWRiMwoweDNiNjZmCjB4MTZhNDUKMHg5NmVhCjB4MTFkYmEKMHgzOWQwNAoweDEyMzliCjB4MTRlNjcKMHgzODM1ZAoweDNhOGYyCjB4MTFiNjAKMHgxNmE0NQoweDFmNjdmCjB4MTZhNDUKMHgxNzNhYwoweDE3M2FjCjB4OTFkYQoweGZlYmIKMHgxMWRiYQoweDM2M2IzCjB4YmQyOAoweDExZGJhCjB4MzgzNWQKMHgxNmE0NQoweDM4NzMxCjB4MTY4M2U=',
 'signature': b'MHg4YjEKMHg4YjEKMHg0NGMKMHg

### Richael's Verification

Now, what Richael will do is:
- Use her own **private** key to decrypt **msg**
- Use Stephen's **public** key to decrypt **signature**
- SHA256 Hash the decrypted **msg** to reproduce the **signature**
- Compare the 2 **signatures** and see if they match:
    - if they do, the message is from Stephen and it is intact.
    - if not, something dodgy happened.

In [22]:
# RSA decode msg
decry_msg = [
    int(i, 16) ** private_key[1] % private_key[0]
    for i in base64.b64decode(message_to_Richael["msg"]).decode("utf8").split("\n")
]
decry_msg = "".join([chr(i) for i in decry_msg])
print(decry_msg)

Hello Richael. Let's meet at 6:25 pm tomorrow at the airport Gate5, Terminal 2 --Stephen :)


In [23]:
# RSA decode sig
decry_sig = [
    int(i, 16) ** public_key_stephen[1] % public_key_stephen[0]
    for i in base64.b64decode(message_to_Richael["signature"]).decode("utf8").split("\n")
]
print(decry_sig)
print("".join([chr(i) for i in decry_sig]))

[50, 50, 101, 100, 99, 57, 50, 57, 100, 57, 50, 49, 49, 100, 49, 49, 48, 50, 102, 55, 97, 102, 56, 56, 98, 56, 53, 101, 97, 52, 55, 50, 99, 57, 101, 54, 52, 54, 50, 57, 48, 55, 52, 57, 56, 53, 99, 98, 97, 57, 52, 54, 48, 55, 53, 50, 54, 100, 102, 102, 51, 48, 50, 57]
22edc929d9211d1102f7af88b85ea472c9e64629074985cba94607526dff3029


In [24]:
# Compare Digital Signature
sha256(
    b"Hello Richael. Let's meet at 6:25 pm tomorrow at the airport Gate5, Terminal 2 --Stephen :)"
).hexdigest() == "".join([chr(i) for i in decry_sig])

True

In [25]:
assert sha256(b"Hello Richael. Let's meet at 6:25").hexdigest() == "".join(
    [chr(i) for i in decry_sig]
), "[Attention]: Dodgy thing happened with this message! Be CAREFULL!"

AssertionError: [Attention]: Dodgy thing happened with this message! Be CAREFULL!

For Joseph, he can also have Stephen's **public key**, intercept and decrypt the signature. However, if now he wants to alter the message ("random" stuff) without being found out, he must alter the Hash ("22edc929d9211d1102f7af88b85ea472c9e64629074985cba94607526dff3029") as well and **make sure they match**. Without decrypting the original message, it is infeasible to generated its Hash. Therefore Joseph has 2 options: alter the message and being found out, or leave message as is.

In [26]:
# alter signature
encry_sig_alter = "\n".join(
    [
        hex((ord(val) ** private_key_stephen[1]) % private_key_stephen[0])
        for val in signature[:10]
    ]
)

# base64 encode
encry_sig_alter = base64.b64encode(encry_sig_alter.encode("utf8"))
encry_sig_alter

b'MHg4YjEKMHg4YjEKMHg0NGMKMHg2NAoweDlmMAoweDViMgoweDhiMQoweDViMgoweDY0CjB4NWIy'

In [27]:
# doesn't match
decry_sig_alter = [
    int(i, 16) ** public_key_stephen[1] % public_key_stephen[0]
    for i in base64.b64decode(encry_sig_alter).decode("utf8").split("\n")
]
print(decry_sig_alter)
print("".join([chr(i) for i in decry_sig_alter]))

[50, 50, 101, 100, 99, 57, 50, 57, 100, 57]
22edc929d9


## Certificate

### Problem:

Last issue: How do we trust those **public** keys received from different identities?

Joseph could totally generate his own key pair and pretend to be Richael. When Richael's friends receive the **public key**, how do they know it truly comes from Richael, rather than Joseph?

Joseph could also generate another pair of keys, sent the **public** key to Richael and claming it is from Stephen. How does Richael know the **public** key is truly from Stephen?

Without verification, Joseph hijacks the information transmission between Richael and Stephen.

### Solution: Certificate Authority (CA)

CA is similar to the driving license issuer, e.g. Police Station or Government. They will verify identity of servers sharing their **public** key to other people and make sure it's genuine. For example:
- Stephen, to avoid being pretended by any other indiviudal like Joseph, must visit CA and show all his proof to convence CA that he is the real Stephen. 
- Once convenced, CA will take over Stephen's **public** key plus other personal information, e.g., name, birthday, address, etc, encrypt them with CA's **private** key, and issue back to Stephen. This is called **Certificate**.
- When sending message to Richael, Stephen actually sends 3 things:
    1. Certificate Issued by CA
    2. Message encrypted by Richael's **public** key
    3. Message Signature encrypted by Stephen's **private** key
- When receiving message from Stephen, Richael does:
    1. Download the **public** key of CA and use it to decrypt **Certificate** included by Stephen. She will obtain the **public** key along with other identity information, for example, name, birthday, address, etc. 
    2. Only if she verified those information and confirm it's Stephen, this **public** key is used to decrypt the **signature**.
    3. Decrypt the message with her own **private** key, and Hash it to reproduce the **signature**.
    4. Compare the 2 **signaure** and see if they match.

In [28]:
# CA generate the key pair
p = random.choice(list(find_prime(1024)))
q = random.choice(list(find_prime(1024)))
assert check_prime(p)
assert check_prime(q)
n = p * q
p, q, n, len(bin(n)), bin(n)

# Euler number
phi = (p - 1) * (q - 1)
phi

e = random.choice(find_prime(phi))
assert check_prime(e)
e

k = modinv(e, phi)[0]
k

public_key_CA, private_key_CA = (n, e), (n, k)
public_key_CA, private_key_CA

((68513, 64123), (68513, 11767))

In [29]:
# Stephen lodge his information
stephen_info = {"pub_key": str(public_key_stephen), "name": "Stephen"}
str(stephen_info)

"{'pub_key': '(2701, 1993)', 'name': 'Stephen'}"

In [30]:
# certificate
certificate = {}
certificate["pub_key"] = base64.b64encode(
    "\n".join(
        [
            hex((ord(val) ** private_key_CA[1]) % private_key_CA[0])
            for val in stephen_info["pub_key"]
        ]
    ).encode("utf8")
)
certificate["name"] = base64.b64encode(
    "\n".join(
        [
            hex((ord(val) ** private_key_CA[1]) % private_key_CA[0])
            for val in stephen_info["name"]
        ]
    ).encode("utf8")
)
certificate

{'pub_key': b'MHgxMDVlNAoweGNhMTYKMHg0MjkyCjB4ODdlOAoweDY0NzcKMHhiZGJjCjB4YWMwOAoweDY0NzcKMHhlZjEzCjB4ZWYxMwoweDEzMDIKMHgzOGI3',
 'name': b'MHg5MTA3CjB4ZTBjYQoweGFiNWMKMHhjN2Y1CjB4MmM4CjB4YWI1YwoweDdiMjM='}

In [31]:
# message send to Richael by Stephen
message_to_Richael = {"cert": certificate, "msg": encry_msg, "signature": encry_sig}

In [32]:
# Richael's Process
# Step 1: download CA's public key
print("CA Public Key: ", public_key_CA)

# Step 2: verify Stephen's certificate with CA's public key
public_key_st = [
    int(i, 16) ** public_key_CA[1] % public_key_CA[0]
    for i in base64.b64decode(message_to_Richael["cert"]["pub_key"])
    .decode("utf8")
    .split("\n")
]
public_key_st = eval("".join([chr(i) for i in public_key_st]))
print("Stepen Public Key: ", public_key_st)

name = [
    int(i, 16) ** public_key_CA[1] % public_key_CA[0]
    for i in base64.b64decode(message_to_Richael["cert"]["name"])
    .decode("utf8")
    .split("\n")
]
print("".join([chr(i) for i in name]))

CA Public Key:  (68513, 64123)
Stepen Public Key:  (2701, 1993)
Stephen


In [33]:
# Step 3: If Richael is Ok with the above info and believe it is Stephen
# RSA decode msg with Richael's private key
decry_msg = [
    int(i, 16) ** private_key[1] % private_key[0]
    for i in base64.b64decode(message_to_Richael["msg"]).decode("utf8").split("\n")
]
decry_msg = "".join([chr(i) for i in decry_msg])
print(decry_msg)

Hello Richael. Let's meet at 6:25 pm tomorrow at the airport Gate5, Terminal 2 --Stephen :)


In [34]:
# RSA decode sig with CA decrypted public key
decry_sig = [
    int(i, 16) ** public_key_st[1] % public_key_st[0]
    for i in base64.b64decode(message_to_Richael["signature"]).decode("utf8").split("\n")
]
print("".join([chr(i) for i in decry_sig]))

22edc929d9211d1102f7af88b85ea472c9e64629074985cba94607526dff3029


In [35]:
# Compare Digital Signature
sha256(decry_msg.encode("utf8")).hexdigest() == "".join([chr(i) for i in decry_sig])

True

Now as to Joseph, if still wants to pretend to be Stephen, would have 3 ways to go:
1. Go to CA and try to convence them that he is the real Stephen, to have his fake **public** key certified, or
2. Manage to have access to CA's **private** key, so he can certify himself, or
3. Become a CA.

Well...

## Reference

- https://blog.vrypan.net/2013/08/28/public-key-cryptography-for-non-geeks/
- https://www.ruanyifeng.com/blog/2013/06/rsa_algorithm_part_one.html
- https://us-cert.cisa.gov/ncas/tips/ST04-018#:~:text=Digital%20signatures%20work%20by%20proving,using%20the%20sender's%20private%20key.